In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/ml_metadata/sdk-metric-parameter-tracking-for-custom-jobs.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/ml_metadata/sdk-metric-parameter-tracking-for-custom-jobs.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_metadata/sdk-metric-parameter-tracking-for-custom-jobs.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>  
</table>

# Vertex AI: Track parameters and metrics for custom training jobs

## Overview

This notebook demonstrates how to track metrics and parameters for Vertex AI custom training jobs, and how to perform detailed analysis using this data.

### Dataset

This example uses the Abalone Dataset. For more information about this dataset please visit: https://archive.ics.uci.edu/ml/datasets/abalone
### Objective

In this notebook, you will learn how to use Vertex AI SDK for Python to:

    * Track training parameters and prediction metrics for a custom training job.
    * Extract and perform analysis for all parameters and metrics within an Experiment.

### Costs 


This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

### Set up your local development environment

**If you are using Colab or Google Cloud Notebooks**, your environment already meets
all the requirements to run this notebook. You can skip this step.

**Otherwise**, make sure your environment meets this notebook's requirements.
You need the following:

* The Google Cloud SDK
* Git
* Python 3
* virtualenv
* Jupyter notebook running in a virtual environment with Python 3

The Google Cloud guide to [Setting up a Python development
environment](https://cloud.google.com/python/setup) and the [Jupyter
installation guide](https://jupyter.org/install) provide detailed instructions
for meeting these requirements. The following steps provide a condensed set of
instructions:

1. [Install and initialize the Cloud SDK.](https://cloud.google.com/sdk/docs/)

1. [Install Python 3.](https://cloud.google.com/python/setup#installing_python)

1. [Install
   virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)
   and create a virtual environment that uses Python 3. Activate the virtual environment.

1. To install Jupyter, run `pip install jupyter` on the
command-line in a terminal shell.

1. To launch Jupyter, run `jupyter notebook` on the command-line in a terminal shell.

1. Open this notebook in the Jupyter Notebook Dashboard.

### Install additional packages

Install additional package dependencies not installed in your notebook environment.

In [1]:
import os

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# Google Cloud Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_GOOGLE_CLOUD_NOTEBOOK:
    USER_FLAG = "--user"

In [15]:
! pip3 install -U tensorflow $USER_FLAG
! python3 -m pip install {USER_FLAG} google-cloud-aiplatform --upgrade
! pip3 install scikit-learn {USER_FLAG}


     |████████████████████████████████| 497.6 MB 7.8 kB/s  eta 0:00:01██████████████████████▉       | 385.5 MB 61.6 MB/s eta 0:00:02     |█████████████████████████▊      | 400.3 MB 65.4 MB/s eta 0:00:02     |█████████████████████████████▊  | 461.8 MB 55.9 MB/s eta 0:00:01
     |████████████████████████████████| 462 kB 33.4 MB/s eta 0:00:01
     |████████████████████████████████| 2.1 MB 38.9 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 4.5 MB/s  eta 0:00:01
     |████████████████████████████████| 42 kB 1.6 MB/s  eta 0:00:01
     |████████████████████████████████| 5.8 MB 50.9 MB/s eta 0:00:01
     |████████████████████████████████| 77 kB 9.1 MB/s  eta 0:00:01
     |████████████████████████████████| 14.5 MB 46.3 MB/s eta 0:00:01
     |████████████████████████████████| 4.5 MB 56.2 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 5.4 MB/s  eta 0:00:01
     |████████████████████████████████| 126 kB 46.9 MB/s eta 0:00:01
     |████████████████████████████████| 

### Restart the kernel

After you install the additional packages, you need to restart the notebook kernel so it can find the packages.

In [4]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Before you begin

### Select a GPU runtime

**Make sure you're running this notebook in a GPU runtime if you have that option. In Colab, select "Runtime --> Change runtime type > GPU"**

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

1. [Enable the Vertex AI API and Compute Engine API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component).

1. If you are running this notebook locally, you will need to install the [Cloud SDK](https://cloud.google.com/sdk).

1. Enter your project ID in the cell below. Then run the cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [33]:
import os

PROJECT_ID = ""

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

E0414 06:09:36.217634390     148 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


Project ID:  vertex-ai-dev


Otherwise, set your project ID here.

In [34]:
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

Set gcloud config to your project ID.

In [35]:
!gcloud config set project $PROJECT_ID

E0414 06:09:40.765771110     148 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


Updated property [core/project].


#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append it onto the name of resources you create in this tutorial.

In [36]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### Authenticate your Google Cloud account

**If you are using Google Cloud Notebooks**, your environment is already
authenticated. Skip this step.

**If you are using Colab**, run the cell below and follow the instructions
when prompted to authenticate your account via oAuth.

**Otherwise**, follow these steps:

1. In the Cloud Console, go to the [**Create service account key**
   page](https://console.cloud.google.com/apis/credentials/serviceaccountkey).

2. Click **Create service account**.

3. In the **Service account name** field, enter a name, and
   click **Create**.

4. In the **Grant this service account access to project** section, click the **Role** drop-down list. Type "Vertex AI"
into the filter box, and select
   **Vertex AI Administrator**. Type "Storage Object Admin" into the filter box, and select **Storage Object Admin**.

5. Click *Create*. A JSON file that contains your key downloads to your
local environment.

6. Enter the path to your service account key as the
`GOOGLE_APPLICATION_CREDENTIALS` variable in the cell below and run the cell.

In [ ]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on Google Cloud Notebooks, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**


When you submit a training job using the Cloud SDK, you upload a Python package
containing your training code to a Cloud Storage bucket. Vertex AI runs
the code from this package. In this tutorial, Vertex AI also saves the
trained model that results from your job in the same bucket. Using this model artifact, you can then
create Vertex AI model and endpoint resources in order to serve
online predictions.

Set the name of your Cloud Storage bucket below. It must be unique across all
Cloud Storage buckets.

You may also change the `REGION` variable, which is used for operations
throughout the rest of this notebook. Make sure to [choose a region where Vertex AI services are
available](https://cloud.google.com/vertex-ai/docs/general/locations#available_regions). You may
not use a Multi-Regional Storage bucket for training with Vertex AI.

In [37]:
BUCKET_URI = "gs://[your-bucket-name]"  # @param {type:"string"}
REGION = "[your-region]"  # @param {type:"string"}

In [38]:
if BUCKET_URI == "" or BUCKET_URI is None or BUCKET_URI == "gs://[your-bucket-name]":
    BUCKET_URI = "gs://" + PROJECT_ID + "-aip-" + TIMESTAMP
    
if REGION == "[your-region]":
    REGION = "us-central1"

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [39]:
! gsutil mb -l $REGION $BUCKET_URI

E0414 06:09:54.546469893     148 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


Creating gs://vertex-ai-dev-aip-20220414060945/...


Finally, validate access to your Cloud Storage bucket by examining its contents:

In [40]:
! gsutil ls -al $BUCKET_URI

E0414 06:09:58.792222871     148 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


### Import libraries and define constants

Import required libraries.


In [41]:
import pandas as pd
from google.cloud import aiplatform
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.python.keras.utils import data_utils

## Initialize Vertex AI and set an _experiment_


Define experiment name.

In [42]:
EXPERIMENT_NAME = ""  # @param {type:"string"}

If EXEPERIMENT_NAME is not set, set a default one below:

In [43]:
if EXPERIMENT_NAME == "" or EXPERIMENT_NAME is None:
    EXPERIMENT_NAME = "my-experiment-" + TIMESTAMP

Initialize the *client* for Vertex AI.

In [44]:
aiplatform.init(
    project=PROJECT_ID,
    location=REGION,
    staging_bucket=BUCKET_URI,
    experiment=EXPERIMENT_NAME,
)

## Tracking parameters and metrics in Vertex AI custom training jobs

This example uses the Abalone Dataset. For more information about this dataset please visit: https://archive.ics.uci.edu/ml/datasets/abalone

In [45]:
!wget https://storage.googleapis.com/download.tensorflow.org/data/abalone_train.csv
!gsutil cp abalone_train.csv {BUCKET_URI}/data/

gcs_csv_path = f"{BUCKET_URI}/data/abalone_train.csv"

E0414 06:10:13.500600241     148 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


--2022-04-14 06:10:14--  https://storage.googleapis.com/download.tensorflow.org/data/abalone_train.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.202.128, 74.125.197.128, 74.125.142.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.202.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 145915 (142K) [text/csv]
Saving to: ‘abalone_train.csv.1’

abalone_train.csv.1 100%[===================>] 142.50K  --.-KB/s    in 0.002s  

2022-04-14 06:10:14 (60.2 MB/s) - ‘abalone_train.csv.1’ saved [145915/145915]



E0414 06:10:14.639959432     148 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


Copying file://abalone_train.csv [Content-Type=text/csv]...
/ [1 files][142.5 KiB/142.5 KiB]                                                
Operation completed over 1 objects/142.5 KiB.                                    


### Create a managed tabular dataset from a CSV

A Managed dataset can be used to create an AutoML model or a custom model. 

In [46]:
ds = aiplatform.TabularDataset.create(display_name="abalone", gcs_source=[gcs_csv_path])

ds.resource_name

Creating TabularDataset


INFO:google.cloud.aiplatform.datasets.dataset:Creating TabularDataset


Create TabularDataset backing LRO: projects/931647533046/locations/us-central1/datasets/7320931047779074048/operations/5805978366016225280


INFO:google.cloud.aiplatform.datasets.dataset:Create TabularDataset backing LRO: projects/931647533046/locations/us-central1/datasets/7320931047779074048/operations/5805978366016225280


TabularDataset created. Resource name: projects/931647533046/locations/us-central1/datasets/7320931047779074048


...

To use this TabularDataset in another session:


INFO:google.cloud.aiplatform.datasets.dataset:To use this TabularDataset in another session:


ds = aiplatform.TabularDataset('projects/931647533046/locations/us-central1/datasets/7320931047779074048')


INFO:google.cloud.aiplatform.datasets.dataset:ds = aiplatform.TabularDataset('projects/931647533046/locations/us-central1/datasets/7320931047779074048')


'projects/931647533046/locations/us-central1/datasets/7320931047779074048'

### Write the training script

Run the following cell to create the training script that is used in the sample custom training job.

In [47]:
%%writefile training_script.py

import pandas as pd
import argparse
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

parser = argparse.ArgumentParser()
parser.add_argument('--epochs', dest='epochs',
                    default=10, type=int,
                    help='Number of epochs.')
parser.add_argument('--num_units', dest='num_units',
                    default=64, type=int,
                    help='Number of unit for first layer.')
args = parser.parse_args()
# uncomment and bump up replica_count for distributed training
# strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()
# tf.distribute.experimental_set_strategy(strategy)

col_names = ["Length", "Diameter", "Height", "Whole weight", "Shucked weight", "Viscera weight", "Shell weight", "Age"]
target = "Age"

def aip_data_to_dataframe(wild_card_path):
    return pd.concat([pd.read_csv(fp.numpy().decode(), names=col_names)
                      for fp in tf.data.Dataset.list_files([wild_card_path])])

def get_features_and_labels(df):
    return df.drop(target, axis=1).values, df[target].values

def data_prep(wild_card_path):
    return get_features_and_labels(aip_data_to_dataframe(wild_card_path))


model = tf.keras.Sequential([layers.Dense(args.num_units), layers.Dense(1)])
model.compile(loss='mse', optimizer='adam')

model.fit(*data_prep(os.environ["AIP_TRAINING_DATA_URI"]),
          epochs=args.epochs ,
          validation_data=data_prep(os.environ["AIP_VALIDATION_DATA_URI"]))
print(model.evaluate(*data_prep(os.environ["AIP_TEST_DATA_URI"])))

# save as Vertex AI Managed model
tf.saved_model.save(model, os.environ["AIP_MODEL_DIR"])

Overwriting training_script.py


### Launch a custom training job and track its trainig parameters on Vertex AI ML Metadata

In [48]:
job = aiplatform.CustomTrainingJob(
    display_name="train-abalone-dist-1-replica",
    script_path="training_script.py",
    container_uri="us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-8:latest",
    requirements=["gcsfs==0.7.1"],
    model_serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-8:latest",
)

Start a new experiment run to track training parameters and start the training job. Note that this operation will take around 10 mins.

In [49]:
aiplatform.start_run("custom-training-run-1")  # Change this to your desired run name
parameters = {"epochs": 10, "num_units": 64}
aiplatform.log_params(parameters)

model = job.run(
    ds,
    replica_count=1,
    model_display_name="abalone-model",
    args=[f"--epochs={parameters['epochs']}", f"--num_units={parameters['num_units']}"],
)

E0414 06:11:18.327794366     148 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


Training script copied to:
gs://vertex-ai-dev-aip-20220414060945/aiplatform-2022-04-14-06:11:18.692-aiplatform_custom_trainer_script-0.1.tar.gz.


INFO:google.cloud.aiplatform.utils.source_utils:Training script copied to:
gs://vertex-ai-dev-aip-20220414060945/aiplatform-2022-04-14-06:11:18.692-aiplatform_custom_trainer_script-0.1.tar.gz.


Training Output directory:
gs://vertex-ai-dev-aip-20220414060945/aiplatform-custom-training-2022-04-14-06:11:18.806 


INFO:google.cloud.aiplatform.training_jobs:Training Output directory:
gs://vertex-ai-dev-aip-20220414060945/aiplatform-custom-training-2022-04-14-06:11:18.806 


...

INFO:google.cloud.aiplatform.training_jobs:CustomTrainingJob projects/931647533046/locations/us-central1/trainingPipelines/5752946652465856512 current state:
PipelineState.PIPELINE_STATE_RUNNING


CustomTrainingJob run completed. Resource name: projects/931647533046/locations/us-central1/trainingPipelines/5752946652465856512


INFO:google.cloud.aiplatform.training_jobs:CustomTrainingJob run completed. Resource name: projects/931647533046/locations/us-central1/trainingPipelines/5752946652465856512


Model available at projects/931647533046/locations/us-central1/models/5805513753634013184


INFO:google.cloud.aiplatform.training_jobs:Model available at projects/931647533046/locations/us-central1/models/5805513753634013184


### Deploy Model and calculate prediction metrics

Deploy model to Google Cloud. This operation will take 10-20 mins.

In [50]:
endpoint = model.deploy(machine_type="n1-standard-4")

Creating Endpoint


INFO:google.cloud.aiplatform.models:Creating Endpoint


Create Endpoint backing LRO: projects/931647533046/locations/us-central1/endpoints/2817960341059665920/operations/9180300386823569408


INFO:google.cloud.aiplatform.models:Create Endpoint backing LRO: projects/931647533046/locations/us-central1/endpoints/2817960341059665920/operations/9180300386823569408


Endpoint created. Resource name: projects/931647533046/locations/us-central1/endpoints/2817960341059665920


...

INFO:google.cloud.aiplatform.models:Deploying model to Endpoint : projects/931647533046/locations/us-central1/endpoints/2817960341059665920


Deploy Endpoint model backing LRO: projects/931647533046/locations/us-central1/endpoints/2817960341059665920/operations/970238266127155200


INFO:google.cloud.aiplatform.models:Deploy Endpoint model backing LRO: projects/931647533046/locations/us-central1/endpoints/2817960341059665920/operations/970238266127155200


Endpoint model deployed. Resource name: projects/931647533046/locations/us-central1/endpoints/2817960341059665920


INFO:google.cloud.aiplatform.models:Endpoint model deployed. Resource name: projects/931647533046/locations/us-central1/endpoints/2817960341059665920


Once model is deployed, perform online prediction using the `abalone_test` dataset and calculate prediction metrics.

Prepare the prediction dataset.

In [51]:
def read_data(uri):
    dataset_path = data_utils.get_file("abalone_test.data", uri)
    col_names = [
        "Length",
        "Diameter",
        "Height",
        "Whole weight",
        "Shucked weight",
        "Viscera weight",
        "Shell weight",
        "Age",
    ]
    dataset = pd.read_csv(
        dataset_path,
        names=col_names,
        na_values="?",
        comment="\t",
        sep=",",
        skipinitialspace=True,
    )
    return dataset


def get_features_and_labels(df):
    target = "Age"
    return df.drop(target, axis=1).values, df[target].values


test_dataset, test_labels = get_features_and_labels(
    read_data(
        "https://storage.googleapis.com/download.tensorflow.org/data/abalone_test.csv"
    )
)

40960/37298 [================================] - 0s 0us/step


Perform online prediction.

In [52]:
prediction = endpoint.predict(test_dataset.tolist())
prediction

Prediction(predictions=[[11.6028872], [12.3952169], [8.24559116], [11.3346586], [9.43768215], [11.5727301], [7.98459816], [11.639739], [12.7643318], [14.5872507], [10.3248339], [8.74010944], [11.2311773], [11.1727209], [11.1439667], [12.1104794], [9.94023323], [10.9762335], [9.31659889], [13.80128], [11.9577532], [8.13747692], [8.23648262], [11.8872728], [10.0421305], [10.3367939], [8.18902874], [8.0494833], [8.57339859], [10.8826323], [6.7982707], [12.5238972], [8.70351124], [7.27030611], [9.02113247], [9.98990345], [12.4704351], [8.04651737], [10.8362141], [10.2476482], [10.0300989], [13.2524261], [7.59112692], [6.77227354], [10.3543921], [7.05193758], [10.4748735], [8.67632198], [10.0631952], [8.344594], [8.69908428], [10.6593065], [12.1251993], [9.31572056], [11.805316], [14.4868183], [11.9081068], [11.5391836], [11.4519281], [9.31171894], [12.294261], [10.967762], [12.1215305], [9.33791256], [11.0665979], [8.54867458], [11.6248159], [6.25234652], [11.730381], [12.6104813], [9.1661

Calculate and track prediction evaluation metrics.

In [53]:
mse = mean_squared_error(test_labels, prediction.predictions)
mae = mean_absolute_error(test_labels, prediction.predictions)

aiplatform.log_metrics({"mse": mse, "mae": mae})

### Extract all parameters and metrics created during this experiment.

In [54]:
aiplatform.get_experiment_df()

,experiment_name,run_name,param.num_units,param.epochs,metric.mse,metric.mae
0,my-experiment-20220414060945,custom-training-run-1,64.0,10.0,7.276731,1.93631


### View data in the Cloud Console

Parameters and metrics can also be viewed in the Cloud Console. 


In [ ]:
print("Vertex AI Experiments:")
print(
    f"https://console.cloud.google.com/ai/platform/experiments/experiments?folder=&organizationId=&project={PROJECT_ID}"
)

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:
Training Job
Model
Cloud Storage Bucket

*   Training Job
*   Model
* Endpoint
* Cloud Storage Bucket


In [56]:
# Warning: Setting this to true will delete everything in your bucket
delete_bucket = False

# Delete the training job
job.delete()

# Undeploy model from endpoint
endpoint.undeploy_all()

# Delete the endpoint
endpoint.delete()

# Delete the model
model.delete()


if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI

Undeploying Endpoint model: projects/931647533046/locations/us-central1/endpoints/2817960341059665920


INFO:google.cloud.aiplatform.models:Undeploying Endpoint model: projects/931647533046/locations/us-central1/endpoints/2817960341059665920


Undeploy Endpoint model backing LRO: projects/931647533046/locations/us-central1/endpoints/2817960341059665920/operations/6811406982826688512


INFO:google.cloud.aiplatform.models:Undeploy Endpoint model backing LRO: projects/931647533046/locations/us-central1/endpoints/2817960341059665920/operations/6811406982826688512


Endpoint model undeployed. Resource name: projects/931647533046/locations/us-central1/endpoints/2817960341059665920


...

Delete Model  backing LRO: projects/931647533046/locations/us-central1/operations/3066663892668121088


INFO:google.cloud.aiplatform.base:Delete Model  backing LRO: projects/931647533046/locations/us-central1/operations/3066663892668121088


Model deleted. . Resource name: projects/931647533046/locations/us-central1/models/5805513753634013184


INFO:google.cloud.aiplatform.base:Model deleted. . Resource name: projects/931647533046/locations/us-central1/models/5805513753634013184
E0414 06:31:39.896580784     148 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


Removing gs://vertex-ai-dev-aip-20220414060945/aiplatform-custom-training-2022-04-14-06:11:18.806/dataset-7320931047779074048-tables-2022-04-14T06:11:19.284460Z/training-00000-of-00004.csv#1649916789490489...
Removing gs://vertex-ai-dev-aip-20220414060945/aiplatform-2022-04-14-06:11:18.692-aiplatform_custom_trainer_script-0.1.tar.gz#1649916678779145...
Removing gs://vertex-ai-dev-aip-20220414060945/aiplatform-custom-training-2022-04-14-06:11:18.806/dataset-7320931047779074048-tables-2022-04-14T06:11:19.284460Z/test-00000-of-00004.csv#1649916790661000...
Removing gs://vertex-ai-dev-aip-20220414060945/aiplatform-custom-training-2022-04-14-06:11:18.806/dataset-7320931047779074048-tables-2022-04-14T06:11:19.284460Z/test-00001-of-00004.csv#1649916790659298...
Removing gs://vertex-ai-dev-aip-20220414060945/aiplatform-custom-training-2022-04-14-06:11:18.806/dataset-7320931047779074048-tables-2022-04-14T06:11:19.284460Z/training-00001-of-00004.csv#1649916789489983...
Removing gs://vertex-ai-de